In [19]:
import pandas as pd
data=pd.read_csv("data/grocery_chain_data.csv")
data.head()

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,-1.56,111
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22


In [20]:
data['transaction_date'] = pd.to_datetime(data['transaction_date'])




In [21]:
daily_demand = data.groupby(
    ['transaction_date', 'product_name']
)['quantity'].sum().reset_index()

daily_demand.head()


,transaction_date,product_name,quantity
0,2023-08-07,Carrots,1.0
1,2023-08-07,Pasta,1.0
2,2023-08-08,Cheese,5.0
3,2023-08-08,Onions,5.0
4,2023-08-08,Orange Juice,4.0


In [22]:
daily_demand['day'] = daily_demand['transaction_date'].dt.day
daily_demand['month'] = daily_demand['transaction_date'].dt.month
daily_demand['year'] = daily_demand['transaction_date'].dt.year


In [23]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
daily_demand['product_encoded'] = le.fit_transform(
    daily_demand['product_name']
)


In [24]:
X = daily_demand[['day', 'month', 'year', 'product_encoded']]
y = daily_demand['quantity']


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [26]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [27]:
y_pred = model.predict(X_test)


In [28]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 1.3305690170044941


In [29]:
comparison = X_test.copy()
comparison['Actual Demand'] = y_test.values
comparison['Predicted Demand'] = y_pred

comparison.head()


,day,month,year,product_encoded,Actual Demand,Predicted Demand
1571,17,4,2025,16,3.0,3.169092
1159,13,11,2024,3,2.0,3.248901
799,24,6,2024,15,1.0,3.262733
1530,2,4,2025,17,4.0,3.063415
1322,19,1,2025,11,3.0,3.150926


In [30]:
daily_demand['product_name'].unique()


array(['Carrots', 'Pasta', 'Cheese', 'Onions', 'Orange Juice', 'Yogurt',
       'Chicken Breast', 'Eggs', 'Bananas', 'Bread', 'Rice', 'Salmon',
       'Cereal', 'Tomatoes', 'Ground Beef', 'Milk', 'Potatoes', 'Apples'],
      dtype=object)

In [31]:
from datetime import datetime, timedelta

future_date = datetime(2026,2,14)

day = future_date.day
month = future_date.month
year = future_date.year


In [32]:
product_name = "Apples"   # change if you want
product_encoded = le.transform([product_name])[0]


In [33]:
import numpy as np

future_input = np.array([[day, month, year, product_encoded]])


In [34]:
future_demand = model.predict(future_input)

print(f"Predicted demand for {product_name} on {future_date.date()} is approximately {future_demand[0]:.2f} units")


Predicted demand for Apples on 2026-02-14 is approximately 3.11 units


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [35]:
!pip freeze > reqirements.txt